# Stuff to make the code works

In [36]:
# Importing all the stuff
import os
import pandas as pd
import numpy as np
import networkx as nx
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import  MaximumLikelihoodEstimator
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score

In [37]:
df = pd.read_csv(f'data{os.sep}heart.csv')
df = df[~(df['Cholesterol'] == 0) & ~(df['RestingBP'] == 0)]
df['ExerciseAngina'] = df['ExerciseAngina'].apply(
    lambda x: False if x == 'N' else True)
df['HeartDisease'] = df['HeartDisease'].apply(
    lambda x: False if x == 0 else True)
df['FastingBS'] = df['FastingBS'].apply(lambda x: False if x == 0 else True)
df["Age"] = pd.cut(x=df["Age"], bins=[20, 30, 40, 50, 60, 70, 80], labels=[
                   "20-30", "30-40", "40-50", "50-60", "60-70", "70-80"])
df["RestingBP"] = pd.cut(x=df["RestingBP"], bins=[90, 120, 140, np.Inf], labels=[
                         "normal", "high", "very-high"])
df["Cholesterol"] = pd.cut(x=df["Cholesterol"], bins=[
                           0, 200, 240, np.Inf], labels=["optimal", "borderline", "high"])
df["MaxHR"] = pd.qcut(x=df["MaxHR"], q=4, labels=["low", "medium", "high", "very-high"])
df["Oldpeak"] = pd.cut(x=df["Oldpeak"], bins=[-np.Inf, 0.5, 1, 2, np.Inf], labels=[
                       "<=0.5", "0-5-1", "1-2", "2+"])

In [38]:
target_variable = "HeartDisease"
X, y = df.drop(columns=target_variable), df[target_variable]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, shuffle=True)
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

# 4. Naïve Bayesian network

A [Naïve Bayesian classifier](https://www.ibm.com/topics/naive-bayes) is a specific type of classifier based on a Naïve Bayesian network. It is characterized by its semplicity, and it is built upond a simple assumption: all the features are conditional independent if the target feature is given.

They are widely used in Machine Learning thanks to their computational efficiency and their ease of implementation. They can also be expanded with other connections, and we will see that using an Hill Climbing algorithm in the next chapter.

Creating a Naïve Bayesian network in `pgmpy` is quite simple: once it is clear which are the features and which is the target variable, a simple list comphrension can build the network structure

In [39]:
network = [(target_variable, x) for x in df.columns[:-1]]
naive_bayesian = BayesianNetwork(network)

In [ ]:
#Doesn't work on windows!
pos = nx.nx_agraph.graphviz_layout(network, prog="dot")
nx.draw(network, pos, with_labels=True, node_size=1000,
        font_size=8, arrowsize=20, alpha=0.8)

Once the structure of the network is built, the next step is to let it learn the parameters (_CPDs_) from the dataset. The choice [laied on two algorithm](https://towardsdatascience.com/maximum-likelihood-vs-bayesian-estimation-dd2eb4dfda8a):

* **Maximum Likelihood Estimation**: Given the likelihood function $\mathcal{L}(\theta | D) = f(D | \theta) = \prod{}_{i=1}^{N}f(x_i | \theta)$, the MLE algorithm tries to fit the parameter $\theta$ that maximize the likelihood function:
$$ \hat{\theta} = \argmax_\theta \prod_{i=1}^N f(x_i | \theta) = \frac{\partial}{\partial \theta} \prod_{i=1}^N f(x_i|\theta)$$ 
Since computing the derivatives of many products can get really complex, in the real world it is used the _log likelihood function_. This is done because the logs of productus is the sum of the logs, and that's simply the computation, and also the $\argmax$ of a function doens't change if we are applying the log, since it is monotonic. So the real formula that is computed is
$$\text{log likelihood} : \int(\theta) = \ln \prod_{i=1}^Nf(x_i | \theta) = \sum_{i=1}^N \ln f(x_i | \theta)$$
Therefore,
$$ \hat{\theta} = \argmax_\theta \int (\theta)$$

This algorithm works the best if the dataset is big, since its outcome solely depends on the _observed_ data. It is also adviced when there is uncertainty about the prior.

* **Bayesian Estimation**: The equation used for Bayesian estimation is the following:
$$ \overbrace{\mathbb{P}(\theta | D)}^\text{posterior distribution} = \frac{\overbrace{\mathbb{P}(D|\theta)}^{\text{likelihood function}} \overbrace{\mathbb{P}(\theta)}^{\text{prior distribution}}}{\int \mathbb{P}(D|\theta) \mathbb{P}(\theta) d \theta}$$
the formula is quite similar to the *Bayes' theorem*, but instead of working with numerical value it uses models and pdfs. The Bayesian estimator tries to compute a distribution over the parameter space, called *posteriod pdf*, and denoted as $\mathbb{P}(\theta | D)$. This distribution represents how strongly we believe each parameter values is the one that generated our data, after taking into account both the observed data and the prior knowledge.

The bayesian estimations works the best if the priors of the networks *makes sense*.


Since we are working on a naïve network, the priors are probably not describing well how each feature affects each other, and we have proceeded using the **Maximum Likelihood Estimation**

To test the performance of the network, we have used KFold to compute an average of the roc_auc score given different permutations of the dataset, to get a more precise idea on the performances

In [41]:
kfold = KFold(10, shuffle=True, random_state=42)

def bayesian_kfold(kfold, model):
    score = []
    for train, test in kfold.split(df):
        model.cpds = []
        model.fit(df.iloc[train, :], estimator=MaximumLikelihoodEstimator)
        y_pred = model.predict(df.iloc[test, :-1])
        score.append(roc_auc_score(df.iloc[test, -1], y_pred[target_variable]))
    return sum(score) / len(score)

In [42]:
%%capture
roc_auc_value = bayesian_kfold(kfold, naive_bayesian)

In [43]:
print("The roc_auc score for the naive bayesian network is {}".format(roc_auc_value))

The roc_auc score for the naive bayesian network is 0.8413832548043075


# Conclusion
With the Naïve Bayesian classifier we get an overall good roc_auc score and we mantain a good computational performace. Is it possibile to improve the network? Does it make sense to add others causal link in between features?